# Quantitative Momentum Strategy Adapted From CodeAcademy (Nick McCullum)

"'Momentum investing' means investing in the stocks that have increased in price the most.

For this project, I am going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, I will calculate recommended trades for an equal-weight portfolio of these 50 stocks." - Nick McCullum

In Nick McCullum's data, there should be a WLTW stock. However, I deleted it cause there was no data stored about it. As a result, there would be a KeyError whenever I ran it. 

## Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from apitoken import IEX_CLOUD_API_TOKEN

## Making Our First API Call

In [3]:
symbol = 'GOOGL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data



{'companyName': 'Alphabet Inc',
 'marketcap': 1875486365982,
 'week52high': 3067.64,
 'week52low': 1866.27,
 'week52highSplitAdjustOnly': 3116.97,
 'week52lowSplitAdjustOnly': 1888.84,
 'week52change': 0.4709000841204555,
 'sharesOutstanding': 306160628,
 'float': 0,
 'avg10Volume': 2347327,
 'avg30Volume': 1866659,
 'day200MovingAvg': 2914.17,
 'day50MovingAvg': 2883.66,
 'employees': 140350,
 'ttmEPS': 106.9,
 'ttmDividendRate': 0,
 'dividendYield': 0,
 'nextDividendDate': '',
 'exDividendDate': '',
 'nextEarningsDate': '2022-01-26',
 'peRatio': 26.10044220125659,
 'beta': 1.263943770299738,
 'maxChangePercent': 10.62193566063445,
 'year5ChangePercent': 2.2390144847697333,
 'year2ChangePercent': 0.8684471825980106,
 'year1ChangePercent': 0.4505378423726611,
 'ytdChangePercent': -0.08314481167298834,
 'month6ChangePercent': -0.021140230436565745,
 'month3ChangePercent': -0.08765457039764595,
 'month1ChangePercent': -0.09309612872117505,
 'day30ChangePercent': -0.09483676730083544,
 'd

## Parsing Our API Call

In [4]:
data['year2ChangePercent']

0.8684471825980106

## Executing A Batch API Call & Building Our DataFrame

In [5]:
my_columns = ['Ticker','Stock Price','Year 2 Change Percent', 'Number of Shares to Buy']

dataFrame = pd.DataFrame(columns = my_columns) # two dimensional 
dataFrame

# will yield n number of chunks that forms a list
def chunks(lst, n):
    for i in range(0,len(lst),n):
        yield lst[i:i + n]
    
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
dataFrame = pd.DataFrame(columns = my_columns)


for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    # print(symbol_string.split(','))
    # print(data)
    for symbol in symbol_string.split(','):
        dataFrame = dataFrame.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['price'],
                    data[symbol]['stats']['year2ChangePercent'],
                    'N/A'
                ],
                index = dataFrame.columns
            ),
            ignore_index = True
        )

dataFrame

,Ticker,Stock Price,Year 2 Change Percent,Number of Shares to Buy
0,A,140.70,0.600005,N/A
1,AAL,16.01,-0.423323,N/A
2,AAP,232.99,0.695906,N/A
3,AAPL,176.48,1.219472,N/A
4,ABBV,138.05,0.841131,N/A
...,...,...,...,...
499,YUM,123.56,0.210173,N/A
500,ZBH,122.60,-0.186717,N/A
501,ZBRA,502.19,0.9966,N/A
502,ZION,68.36,0.561729,N/A


## Removing Low-Momentum Stocks

In [6]:
dataFrame.sort_values('Year 2 Change Percent', ascending = False, inplace = True) # inplace modifies the orginal pands dataframe
dataFrame = dataFrame[:50]
dataFrame.reset_index(inplace = True)
dataFrame

,index,Ticker,Stock Price,Year 2 Change Percent,Number of Shares to Buy
0,78,CARR,46.47,3.004005,N/A
1,345,NVDA,230.80,2.776376,N/A
2,275,LB,83.82,2.49429,N/A
3,179,FCX,36.74,2.273933,N/A
4,299,MCHP,74.27,1.910389,N/A
5,23,ALB,212.74,1.678295,N/A
6,489,WST,397.65,1.581258,N/A
7,148,DVN,51.78,1.544462,N/A
8,385,PWR,102.20,1.486415,N/A
9,195,FTNT,282.76,1.448141,N/A


## Calculating the Number of Shares to Buy

In [7]:
def portfolio():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio ")
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again ')
        portfolio_size = input("Enter the size of your portfolio ")
        
portfolio()
print(portfolio_size)

Enter the size of your portfolio 1000000
1000000


In [8]:
position_size = float(portfolio_size) / len(dataFrame.index)
# print(position_size)
for i in range(0, len(dataFrame)):
    dataFrame.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / dataFrame.loc[i, 'Stock Price'])
    
dataFrame

/Users/jiyoonsung/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Stock Price,Year 2 Change Percent,Number of Shares to Buy
0,78,CARR,46.47,3.004005,430
1,345,NVDA,230.80,2.776376,86
2,275,LB,83.82,2.49429,238
3,179,FCX,36.74,2.273933,544
4,299,MCHP,74.27,1.910389,269
5,23,ALB,212.74,1.678295,94
6,489,WST,397.65,1.581258,50
7,148,DVN,51.78,1.544462,386
8,385,PWR,102.20,1.486415,195
9,195,FTNT,282.76,1.448141,70


## Building a Better (and More Realistic) Momentum Strategy

In [14]:
hqm_col = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Two-Years Price Return',
    'Two-Years Return Percentile',
    'Six-Months Price Return',
    'Six-Months Return Percentile',
    'Three-Months Price Return',
    'Three-Months Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile'
]

hqm_dataFrame = pd.DataFrame(columns = hqm_col)
# hqm_dataFrame

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataFrame = hqm_dataFrame.append(
            pd.Series(
                [symbol, data[symbol]['price'], 'N/A', data[symbol]['stats']['year2ChangePercent'], 'N/A', data[symbol]['stats']['month6ChangePercent'], 'N/A', data[symbol]['stats']['month3ChangePercent'], 'N/A', data[symbol]['stats']['month1ChangePercent'], 'N/A'],
                index = hqm_col
            ),
            ignore_index = True
        )
        
hqm_dataFrame
    

,Ticker,Price,Number of Shares to Buy,Two-Years Price Return,Two-Years Return Percentile,Six-Months Price Return,Six-Months Return Percentile,Three-Months Price Return,Three-Months Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,140.70,N/A,0.609168,N/A,-0.094447,N/A,-0.124598,N/A,-0.139066,N/A
1,AAL,16.00,N/A,-0.421513,N/A,-0.28088,N/A,-0.189059,N/A,-0.159345,N/A
2,AAP,233.25,N/A,0.691955,N/A,0.08691,N/A,0.019242,N/A,-0.04171,N/A
3,AAPL,176.13,N/A,1.229327,N/A,0.180834,N/A,0.120403,N/A,-0.051277,N/A
4,ABBV,138.95,N/A,0.833775,N/A,0.190089,N/A,0.273537,N/A,0.037944,N/A
...,...,...,...,...,...,...,...,...,...,...,...
499,YUM,125.01,N/A,0.213559,N/A,0.004808,N/A,-0.026099,N/A,-0.115862,N/A
500,ZBH,123.03,N/A,-0.184471,N/A,-0.259742,N/A,-0.176607,N/A,-0.059093,N/A
501,ZBRA,500.85,N/A,1.006706,N/A,-0.101966,N/A,-0.077225,N/A,-0.178568,N/A
502,ZION,69.33,N/A,0.555876,N/A,0.329678,N/A,0.092467,N/A,0.080989,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores, but I am currently unable to find a way to. In the course, he uses the for loop below. However, I am unable to because I cannot support the instances of 'NoneType' and 'float'.

In [30]:
time_periods = ['Two-Years', 'Six-Months', 'Three-Months', 'One-Month']

for row in hqm_dataFrame.index:
    for time_period in time_periods:
        hqm_dataFrame.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataFrame[f'{time_period} Price Return'], hqm_dataFrame.loc[row, f'{time_period} Price Return'])/100
        
for time_period in time_periods:
    print(hqm_dataFrame[f'{time_period} Return Percentile'])
    
hqm_dataFrame
        

TypeError: '<' not supported between instances of 'NoneType' and 'float'

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: